<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [ ]:
#Importing warnings to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [1]:
#ALL imports
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import nltk
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_lg")

In [2]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
#checking out the dataset
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [4]:
#taking a look at the text reviews
yelp['text']

0       BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...
1       Came here for lunch Togo. Service was quick. S...
2       I've been to Vegas dozens of times and had nev...
3       We went here on a night where they closed off ...
4       3.5 to 4 stars\n\nNot bad for the price, $12.9...
5       Tasty, fast casual Latin street food.  The men...
6       This show is absolutely amazing!! What an incr...
7       Came for the Pho and really enjoyed it!  We go...
8       Absolutely the most Unique experience in a nai...
9       Wow. I walked in and sat at the bar for 10 min...
10      We popped in for dinner yesterday with no rese...
11      Thw worst stay ever! So first i ended up payin...
12      Great friendly customer service and quality fo...
13      The food was great!  It was super busy but our...
14      Talk about getting ripped off. They charged us...
15      Girls night out tonight with my kid so we deci...
16      Stopped in here for a few drinks flying out of...
17      This i

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [5]:
#Creating stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [6]:
#creating my tokenizer
def tokenize(doc):
    return [token for token in simple_preprocess(doc) if token not in stopwords]

In [7]:
#Returning the use of my tokenizer
tokenize("This is an example of my tokenizer taking the stop words away from this current sentence, I hope it works well. Thank you!")

['example',
 'tokenizer',
 'taking',
 'stop',
 'words',
 'away',
 'current',
 'sentence',
 'hope',
 'works',
 'well',
 'thank']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [101]:
#setting my data
data = yelp['text']

In [9]:
#Creating my vectorizer and fitting it into the data
vectorizer = CountVectorizer(stop_words='english')

vectorizer.fit(data)

vectorizer.vocabulary_

dtm = vectorizer.transform(data)

In [10]:
dtm.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [11]:
#Grabbing Feature names and putting data into a vector
dtm = pd.DataFrame(dtm.todense(), columns = vectorizer.get_feature_names())

In [12]:
# Combining CountVectorizer with the data 
vect = CountVectorizer(stop_words = 'english', min_df = 0.05, max_df = 0.90)
vect.fit(data)

sparse_dtm = vect.transform(data)

dtm = pd.DataFrame(sparse_dtm.todense(), columns = vect.get_feature_names())

In [13]:
#Vectorized data reviews
dtm

,10,amazing,area,asked,away,awesome,bad,bar,best,better,...,vegas,wait,want,wanted,wasn,way,went,work,worth,years
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,2,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#Settng up TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

#fitting
dtm = tfidf.fit_transform(data)

#Viewing Matrix as a Data Frame
docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
docs.head()

,00,000,00pm,07,10,100,1000,101,10pm,11,...,yuck,yuk,yum,yummy,yup,zero,zone,zoo,zucchini,était
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm)

# Turn it into a DataFrame
dist_df = pd.DataFrame(dist_matrix)

dist_df[0].idxmax(axis=0)

dist_df.shape

(10000, 10000)

In [36]:
dist_df[0]

0       1.000000
1       0.000000
2       0.014419
3       0.000000
4       0.000000
5       0.029653
6       0.003368
7       0.005604
8       0.002290
9       0.032405
10      0.007878
11      0.043679
12      0.000000
13      0.000000
14      0.154955
15      0.000000
16      0.003899
17      0.000000
18      0.019884
19      0.019979
20      0.022204
21      0.002976
22      0.067787
23      0.003576
24      0.000000
25      0.012101
26      0.000000
27      0.000000
28      0.000000
29      0.007354
          ...   
9970    0.030518
9971    0.034013
9972    0.000000
9973    0.064112
9974    0.000000
9975    0.002724
9976    0.023644
9977    0.000000
9978    0.000000
9979    0.006362
9980    0.004053
9981    0.000000
9982    0.021598
9983    0.034879
9984    0.005219
9985    0.017295
9986    0.020994
9987    0.000000
9988    0.011112
9989    0.006509
9990    0.000000
9991    0.000000
9992    0.000000
9993    0.013314
9994    0.022256
9995    0.052587
9996    0.000000
9997    0.0000

In [37]:
dtm

<10000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 376778 stored elements in Compressed Sparse Row format>

In [38]:
#setting up NearestNeigbors
nn = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree')

# Fitting NN with vector
nn.fit(dtm)

/Users/danielcalimayor/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/base.py:217: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [39]:
#Activating Query using kneighbors
nn.kneighbors(dtm)

(array([[0.        , 1.        , 1.        , 1.25325603, 1.25954602],
        [0.        , 1.        , 1.        , 1.17772553, 1.19154405],
        [0.        , 1.        , 1.        , 1.06839889, 1.11038143],
        ...,
        [0.        , 1.        , 1.        , 1.22137603, 1.2229785 ],
        [0.        , 1.        , 1.        , 1.24537508, 1.24737227],
        [0.        , 1.        , 1.        , 1.21437929, 1.22620797]]),
 array([[   0, 6204, 6311, 2131, 3276],
        [   1, 6204, 6311, 7161, 9490],
        [   2, 6311, 6204,  431, 5942],
        ...,
        [9997, 6311, 6204, 7583, 3065],
        [9998, 6311, 6204,   19, 2415],
        [9999, 6311, 6204, 9676, 7279]]))

In [52]:
#creating my own simple review
my_review = ['''I love Chipotle. One hundred percent recommended. Nice Service. also, the bathrooms were very clean and had a really
nice scent. Kind people and they really take care of you! the outside patio was good and the employees will go out of their way
to give you great service.''']

In [53]:
#Transforming my review
new = tfidf.transform(my_review)
new

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [54]:
#combinging reviews with nearest neighbors
nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.14261509, 1.1700888 , 1.21659494]]),
 array([[6204, 6311, 9511, 3044, 3693]]))

In [55]:
#BOOM
data[3044]

'I love the customer service,  the store is always clean and the food is great. I just want to tell the employees at Chipotle keep up the great work your food is awesome. There is always challenges Vegas Restaurants are awesome and friendly. Thank you for the awesome service.'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [56]:
yelp['stars'].head()

0    1
1    4
2    3
3    1
4    4
Name: stars, dtype: int64

In [58]:
pl = yelp['stars']

In [62]:
#Creating Pipeline
vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [63]:
#Fitting Pipeline

pipe.fit(data, pl)

/Users/danielcalimayor/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [70]:
#Preicting the rating on my fake review
pipe.predict(my_review)

array([5])

In [67]:
#Setting up Parameters
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__max_iter':(20, 10, 100)
}

In [68]:
#Setting Up Gridsearch and adding the parameters
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)

In [69]:
grid_search.fit(data, pl)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   44.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  4.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'clf__max_iter': (20, 10, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [71]:
grid_search.best_params_

{'clf__max_iter': 10,
 'vect__max_df': 0.75,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [72]:
#using grid search to predict the rating on my review
grid_search.predict(my_review)

array([5])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [102]:
id2word = corpora.Dictionary(dtm)

TypeError: decoding to str: need a bytes-like object, csr_matrix found

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)